### Pakete laden und Daten einlesen

In [ ]:
import pandas as pd
import re

# Daten einlesen
df = pd.read_excel("bushel_series_DataChallenge_2025-numisdata4-2_TypenEinzeln.xlsx")

### Nicht benötigte Spalten/Zeilen löschen

In [3]:
# Spalten Typ_2 bis Typ_7 löschen, falls vorhanden
for spalte in [f"Typ_{i}" for i in range(2, 8)]:
    if spalte in df.columns:
        df.drop(columns=spalte, inplace=True)

# Spalte 'Type | Reverse design' löschen
df.drop(columns="Type | Reverse design", inplace=True)

# Spalte 'Type | Obverse design' löschen
df.drop(columns="Type | Obverse design", inplace=True)

# Zeilen löschen, wenn 'Findspot' leer ist
df = df[~(df['Findspot'].isna())]

### Spalte Findspot bereinigen

In [4]:
def verarbeite_findspot(zelle_findspot):
    zelle_findspot = str(zelle_findspot).strip()
    klammer_findspot = re.findall(r'\((.*?)\)', zelle_findspot)
    ohne_klammer_findspot = re.sub(r'\s*\(.*?\)', '', zelle_findspot)
    teile_findspot = re.split(r'\s*[|,]\s*', ohne_klammer_findspot.strip())
    teile_findspot = [t for t in teile_findspot if t] + klammer_findspot
    return teile_findspot

# Im DataFrame anwenden
findspot_liste = df['Findspot'].apply(verarbeite_findspot)

# Bestimme maximale Anzahl Spalten
spalten_findspot = [f'Findspot_{i+1}' for i in range(findspot_liste.map(len).max())]

# Liste in Spalten umwandeln und zum DataFrame hinzufügen
df[spalten_findspot] = pd.DataFrame(findspot_liste.tolist(), index=df.index)

# Originalspalte löschen
df.drop(columns='Findspot', inplace=True)

### Spalte Type_1 bereinigen

In [5]:
spalte_typ1="Typ_1"

def verarbeite_typ1_zelle(zelle_typ1):
    if pd.isna(zelle_typ1) or zelle_typ1.strip() == "":
        return []

    # Aufteilen und Duplikate entfernen
    teile_typ1 = [t_typ1.strip() for t_typ1 in zelle_typ1.strip('|').split('|') if t_typ1.strip()]
    unique_teile_typ1 = []
    gesehen_typ1 = set()
    for t_typ1 in teile_typ1:
        if t_typ1 not in gesehen_typ1:
            gesehen_typ1.add(t_typ1)
            unique_teile_typ1.append(t_typ1)

    # Separate Speicherung für Bushel series und Silver unit
    output_typ1_1_2 = [None, None]  # [Typ_1_1, Typ_1_2]
    rest_typ1_1_2 = []

    for eintrag_typ1_1_2 in unique_teile_typ1:
        if eintrag_typ1_1_2 == "Bushel series":
            output_typ1_1_2[0] = "Bushel series"
        elif eintrag_typ1_1_2 == "Silver unit":
            output_typ1_1_2[1] = "Silver unit"
        else:
            rest_typ1_1_2.append(eintrag_typ1_1_2)

    return output_typ1_1_2 + rest_typ1_1_2  # ergibt: [Bushel series?, Silver unit?, rest...]

# Liste erzeugen
typ1_liste = df[spalte_typ1].apply(verarbeite_typ1_zelle)

# Max. Anzahl Elemente
max_typ1 = typ1_liste.map(len).max()

# Neue Spalten erzeugen
spalten_typ1 = [f'{spalte_typ1}_{i+1}' for i in range(max_typ1)]
df[spalten_typ1] = pd.DataFrame(typ1_liste.tolist(), index=df.index)

# Originalspalte löschen
df.drop(columns=spalte_typ1, inplace=True)

# Spalte Typ_1_4 bereinigen
df["Typ_1_4"] = df["Typ_1_4"].replace(
    to_replace=r"\b(Nick|Kellner|Ziegaus)\b\s*", 
    value="", 
    regex=True
).str.strip()

# Zeilen löschen, wenn kein Typ_1_4 vorhanden oder 'Uncertain type' ist
df = df[~(df['Typ_1_4'].isna() | (df['Typ_1_4'].str.strip() == 'Uncertain type'))]

# Kopie von Typ_1_4 erstellen
df['Typ'] = df['Typ_1_4'].copy()

# Funktion zur Aufteilung und Mapping
def verarbeite_typ_zelle(zelle_stamp):
    if pd.isna(zelle_stamp):
        return pd.Series([None, None])  # [Typ, Stamp]
    
    teile_stamp = zelle_stamp.strip().split()
    
    # Wenn 3 Teile (z.B. "BS C 4")
    if len(teile_stamp) == 3 and teile_stamp[0] == 'BS':
        typ_wert = ' '.join(teile_stamp[:2])  # "BS C"
        stamp_wert = teile_stamp[2]          # "4"
    else:
        typ_wert = zelle_stamp.strip()
        stamp_wert = None

    # Mapping für spezielle Fälle
    mapping = {
        'BS 1': 'BS Prototype',
        'BS 2': 'BS A',
        'BS 3': 'BS B',
        'BS 4': 'BS C',
        'BS 5': 'BS D',
        'BS 6': 'BS E',
        'BS 7': 'BS F',
        'BS 8': 'BS G',
        'BS 9': 'BS H'
    }
    
    # Mapping anwenden, falls exakt passend
    typ_wert = mapping.get(typ_wert, typ_wert)
    return pd.Series([typ_wert, stamp_wert])

# Anwenden der Funktion auf Spalte 'Typ'
df[['Typ', 'Stamp']] = df['Typ'].apply(verarbeite_typ_zelle)

### Spalte "Findspot | Remark public" einfügen

In [ ]:
# Neue Excel einlesen (nur benötigte Spalten)
df_remark = pd.read_excel("export_Numismatic object_30.5.2025-numisdata4.xlsx", usecols=["Id", "Findspot | Remark public"])

# Spalte umbenennen
df_remark.rename(columns={"Findspot | Remark public": "Remark public"}, inplace=True)

# Merge mit bestehendem DataFrame über die Spalte 'Id'
df = df.merge(df_remark, on="Id", how="left")  # 'left', um alle Zeilen aus df zu behalten

# Inhalt der neuen Spalte "Remark public" bereinigen
def bereinige_remark_public(eintrag_remark):
    if pd.isna(eintrag_remark):
        return eintrag_remark
    eintrag_remark = eintrag_remark.strip()  # führende/trailing Leerzeichen entfernen
    
    # Mapping für die Ersetzungen
    ersetzungen_remark = {
        "Cult places": "Cult place",
        "Cult placeBrandopferplatz": "Cult place",
        "Settlment": "Settlement",
        "SettlementViereckschanze": "Settlement",
        "Single find| Settlement": "Single find"
    }

      
    # Ersetze falls im Dictionary vorhanden
    return ersetzungen_remark.get(eintrag_remark, eintrag_remark)  # wenn nicht in ersetzungen: bleibt original

# Anwenden auf die Spalte
df["Remark public"] = df["Remark public"].apply(bereinige_remark_public)


### Fehlende Werte im df mit NaN ersetzen

In [7]:
df = df.where(pd.notnull(df), "NaN")

### Neuen Datensatz speichern

In [ ]:
df.to_csv('bushel_series_DataChallenge_2025-new.csv', index=False)